# Install stuff on Colab and imports

In [ ]:
#!nvidia-smi

In [ ]:
# In case you want to run it on Google Colab, don't forget to install biome.text first

# !pip install -U pip
# !pip install -U git+https://github.com/recognai/biome-text.git@project/profner
# exit(0)

In [ ]:
# !pip install wandb

In [ ]:
# !wandb login

In [4]:
import itertools
import os
from pathlib import Path

from biome.text import Dataset, Pipeline, TrainerConfiguration
from biome.text.hpo import TuneExperiment
from ray import tune
from ray.tune.suggest.hyperopt import HyperOptSearch

# Create training and validation data sets

In [5]:
train_ds = Dataset.from_json("../preprocessing_inference/train_v1.json")
valid_ds = Dataset.from_json("../preprocessing_inference/valid_v1.json")

Using custom data configuration default


Dataset json downloaded and prepared to /home/david/.cache/huggingface/datasets/json/default-6bacbcbff55d5326/0.0.0/70d89ed4db1394f028c651589fcab6d6b28dddcabbe39d3b21b4d41f9a708514. Subsequent calls will reuse this data.


Using custom data configuration default


Dataset json downloaded and prepared to /home/david/.cache/huggingface/datasets/json/default-b2dd006bd782a885/0.0.0/70d89ed4db1394f028c651589fcab6d6b28dddcabbe39d3b21b4d41f9a708514. Subsequent calls will reuse this data.


In [6]:
train_ds.rename_column_("tags_bio", "tags")
valid_ds.rename_column_("tags_bio", "tags")
train_ds.rename_column_("classification_label", "labels")
valid_ds.rename_column_("classification_label", "labels")

In [7]:
train_ds.head()

,raw_text,tokens,tags_bioul,tags,entity_text,labels,file_name
0,Cerramos nuestra querida Radio 😢 Nuestros cola...,"[Cerramos, nuestra, querida, Radio, 😢, Nuestro...","[O, O, O, O, O, O, U-PROFESION, O, U-PROFESION...","[O, O, O, O, O, O, B-PROFESION, O, B-PROFESION...","[colaboradores, conductores]",1,1242399976644325376.txt
1,#OtroEscandalo #HastaCuando \n#DenunciaCCOO #C...,"[#, OtroEscandalo, #, HastaCuando, \n, #, Denu...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",[],0,1242406334802395137.txt
2,¿Es necesario entregar nuestra privacidad a un...,"[¿, Es, necesario, entregar, nuestra, privacid...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",[],0,1242407077278093313.txt
3,Así que estás chimbeando mucho con esos Decret...,"[Así, que, estás, chimbeando, mucho, con, esos...","[O, O, O, O, O, O, O, O, O, O, O, U-PROFESION,...","[O, O, O, O, O, O, O, O, O, O, O, B-PROFESION,...",[Presidente],1,1242407274771030016.txt
4,@FeGarPe79 @escipion_r @LuciaMendezEM Estás MU...,"[@FeGarPe79, @escipion_r, @LuciaMendezEM, Está...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",[],0,1242409866515435520.txt
5,La Generalitat facilitará las videconferencias...,"[La, Generalitat, facilitará, las, videconfere...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",[],0,1242420050167988227.txt
6,“El pánico por coronavirus es injustificado” d...,"[“, El, pánico, por, coronavirus, es, injustif...","[O, O, O, O, O, O, O, O, O, O, U-PROFESION, O,...","[O, O, O, O, O, O, O, O, O, O, B-PROFESION, O,...",[virólogo],1,1242429168505233410.txt
7,La transparencia es necesaria para luchar cont...,"[La, transparencia, es, necesaria, para, lucha...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",[],0,1242448823810654209.txt
8,Ojo con los mensajes que se están lanzando des...,"[Ojo, con, los, mensajes, que, se, están, lanz...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",[],0,1242501570824200194.txt
9,¿Dispones de fundas de plástico cubreasientos ...,"[¿, Dispones, de, fundas, de, plástico, cubrea...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",[],0,1242506209506312193.txt


# Get pretrained word vectors

In [ ]:
!wget https://zenodo.org/record/4449930/files/skipgram_cased.tar.gz

In [ ]:
!tar -xzf skipgram_cased.tar.gz

# Define pipeline/trainer config and HPO search spaces 

In [8]:
# This is the config of a second HPO run, a
# fter a first HPO run we fixed obviously better choices like:
# the weights_file, lowercase_characters, some RNN types and num_layers
profner = {
    "name": "profner",
    "features": {
        "word": {
            "embedding_dim": 300, 
            "weights_file": str(Path("./covid_19_es_twitter_skipgram_cased.zip").absolute()),
            "trainable": True,
        },
        'char': {
            'embedding_dim': tune.choice([32, 64]),
            'lowercase_characters': True,
            'encoder': {
                'bidirectional': True,
                'hidden_size': tune.choice([32, 64]),
                'num_layers': 1,
                'type': 'gru',
            },
            'dropout': tune.uniform(0, 0.5),
        },
    },
    "encoder": {
        "type": tune.choice(["gru", "lstm"]),
        "num_layers": 1,
        "bidirectional": True,
        "hidden_size": tune.choice([256, 512]),
    },
    "head": {
        "type": "ProfNer",
        "classification_labels": ['1', '0'],
        "classification_pooler": {
            "type": "lstm",
            "num_layers": 1,
            "bidirectional": True,
            "hidden_size": tune.choice([64, 128]),
        },
        "ner_feedforward": {
            "activations": ["relu"],
            "dropout": [0],
            "hidden_dims": [128],
            "num_layers": 1,
        },
        "ner_tags": ['O', 'B-PROFESION', 'I-SITUACION_LABORAL', 'I-PROFESION', 'B-SITUACION_LABORAL'],
        "ner_tags_encoding": "BIO",
        "dropout": tune.uniform(0, 0.7),
    },
}

In [10]:
trainer_config = dict(
    optimizer={
        "type": "adamw",
        "lr": tune.loguniform(1e-4, 1e-1),
        "weight_decay": tune.loguniform(1e-3, 1e-1)
    },
    linear_decay=True,
    warmup_steps=tune.randint(0, 200),
    training_size=len(train_ds),
    batch_size=tune.choice([8, 16, 32]),
    num_epochs=tune.choice([4, 5, 6, 7]),
    validation_metric="+valid_ner/f1-measure-overall"
)

# Launch HPO experiment

In [11]:
search_alg = HyperOptSearch(metric="validation_valid_ner/f1-measure-overall", mode="max")

# does not support integers
# from ray.tune.suggest.bayesopt import BayesOptSearch
# search_alg = BayesOptSearch(metric="validation_ner/f1-measure-overall", mode="max")

# ray2.0.0 does not use the gpu ...
# from ray.tune.suggest.hebo import HEBOSearch
# search_alg = HEBOSearch(metric="validation_ner/f1-measure-overall", mode="max")

In [12]:
hpo_experiment = TuneExperiment(
    pipeline_config=profner,
    trainer_config=trainer_config,
    train_dataset=train_ds,
    valid_dataset=valid_ds,
    name="profner_rnn",
    num_samples=1,
    local_dir="tune_runs",
    resources_per_trial={"cpu": 1, "gpu": 0.33},
)

In [ ]:
analysis = tune.run(
    hpo_experiment,
    config=hpo_experiment.config,
    scheduler=tune.schedulers.ASHAScheduler(),
    search_alg=search_alg,
    metric="validation_valid_ner/f1-measure-overall",
    mode="max",
#     progress_reporter=tune.CLIReporter(
#             metric_columns=["best_validation_valid_ner/f1-measure-overall", "best_validation_valid_classification/accuracy"],
#             parameter_columns=["trainer.optimizer.lr"],
#         ),
    progress_reporter=tune.JupyterNotebookReporter(overwrite=True),
    verbose=2,
)

# Train the best model on train+validation data set

See the "profner_rnn" project in wandb (https://wandb.ai/dcfidalgo/profner_rnn), the runs tagged as v3. Best run: *helpful-sun-581*; 0.731, 0.938;

In [ ]:
train_full_ds = Dataset.from_datasets([train_ds, valid_ds])

In [ ]:
profner = {
    "name": "profner",
    "features": {
        "word": {
            "embedding_dim": 300, 
            "weights_file": str(Path("./covid_19_es_twitter_skipgram_cased.zip").absolute()),
            "trainable": True,
        },
        'char': {
            'embedding_dim': 64,
            'lowercase_characters': True,
            'encoder': {
                'bidirectional': True,
                'hidden_size': 64,
                'num_layers': 1,
                'type': 'gru',
            },
            'dropout': 0.008045582408546581
        }
    },
    "encoder": {
        "type": "gru",
        "num_layers": 1,
        "bidirectional": True,
        "hidden_size": 512,
    },
    "head": {
        "type": "ProfNer",
        "classification_labels": train_ds.unique("labels"),
        "classification_pooler": {
            "type": "lstm",
            "num_layers": 1,
            "bidirectional": True,
            "hidden_size": 64,
        },
        "ner_feedforward": {
            "activations": ["relu"],
            "dropout": [0],
            "hidden_dims": [128],
            "num_layers": 1,
        },
        "ner_tags": list(set(itertools.chain.from_iterable(train_ds["tags"]))),
        "ner_tags_encoding": "BIO",
        "dropout": 0.43890376017930377,
    },
}

In [ ]:
trainer = TrainerConfiguration(
    optimizer={
        "type": "adamw",
        "lr": 0.0008738642587749882,
        "weight_decay": 0.0015643023992246586,
    },
    linear_decay=True,
    warmup_steps=113,
    batch_size=8,
    num_epochs=5,
    validation_metric="+valid_ner/f1-measure-overall",
    random_seed=4,  # best value from a quick manual sweep
)

In [ ]:
pipeline = Pipeline.from_config(profner)

In [ ]:
pipeline.train(
    output="final_rnn_model",
    training=train_full_ds,
    trainer=trainer,
)